In [1]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os
import pandas as pd
!pip install tiktoken

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9572\3627484799.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('law_dict_4.csv', index_col=0) #단어사전 불러오기


NameError: name 'pd' is not defined

In [13]:
"""토큰 계산 코드"""
import tiktoken
df = pd.read_csv('law_dict_4.csv', index_col=0) #단어사전 불러오기
df.columns = ['title', 'text'] #순서대로 칼럼 넣기

#  cl100k_base는 토크나이저다. 임베딩과는 큰 관련이 없다.
tokenizer = tiktoken.get_encoding("cl100k_base")

# 소요되는 토큰을 각 칼럼에 저장하기. apply 함수가 쓰였다!
df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))


ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [ ]:
# 히스토그램으로 그려서 보기
df.n_tokens.hist()

In [ ]:
"""본격적인 임베딩 시작"""

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv('../../.env'))  #.env 파일 찾기
from openai import OpenAI
import openai
client = OpenAI()

In [ ]:
# 됐으면 바로 다 집어 넣어 ~
# df라는 데이터 프레임에 칼럼 embeddings를 넣을 건데 그건 각각의 임베딩 값이 된다.
df['embeddings'] = df.text.apply(lambda x: client.embeddings.create(input=x, model='text-embedding-ada-002').data[0].embedding)
df.to_csv('law_dict_inclue_embeddings.csv') #embeddings칼럼이 추가된 df를 따로 저장해둔다.
df.head() #견본을 살펴본다. 잘됐나?

In [ ]:
"""이제부터는 데이터프레임을 임베딩값 연산 편하게 apply해서 np.array로 변환해줄거임 그게 연산 최고"""
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

In [ ]:
#아까 저장했던 embeedings칼럼이 들어간 df를 df으로 다시 불러온다. index_col=0으로 두어 쓸데없는 칼럼 인덱스 추가 안한다.
df=pd.read_csv('law_dict_inclue_embeddings.csv', index_col=0)

# 임베딩스 열의 임베딩 값들을 np.array를 통해 다시 배열해 준다 > 연산에 효율성이 좋다. 여기서 eval 쓰는건 이거 파이썬 값인걸
# 가끔 인지 못하는 경우가 있어서 확실하게 eval 해주는 것.
df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)

#견본을 살펴보자 array 잘 됐나?
df.head()

In [ ]:
#create_context라는 함수는 question과 df, max_len을 인자로 가지는데 여기서 max_len은 1800으로 정해져 있다.
def create_context(
    question, df, max_len=1800
):
    """
    데이터 프레임으로부터 가장 유사한 문맥을 찾아서 질문에 대한 문맥을 만든다.
    """

    # 질문으로부터 임베딩 값을 만든다.
    q_embeddings = client.embeddings.create(input=question, model='text-embedding-ada-002').data[0].embedding
    

    # df상의 임베딩 값들과 qestion의 임베딩 값들의 코사인 유사도를 통해 distance를 계산한다.
    # cosine similarity: 문자열끼리 유사도 비교하는 것.
    df["distances"] = df["embeddings"].apply(lambda x: cosine(q_embeddings, x))


    returns = []
    cur_len = 0

    # 거리순으로 정렬하고, 문맥이 너무 길어지기 전까지만 텍스트를 문맥에 추가한다.
    #distances 칼럼의 값들을 거리순으로 정렬시킨다.
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        

        # 현재 토큰 수에 텍스트 길이를 더한다.
        cur_len += row['n_tokens'] + 4
        
        # 그렇게 더해진 현재 토큰수는 곧 문맥이될 터인데, 그것이 max_len으로 설정한 값보다 길어지면, break하고
        # 밑에 바로 return으로 넘어간다.
        if cur_len > max_len:
            break
        

        # 하지만 그렇지 않다면, 문맥에 추가한다. else안 쓴거는 그렇게 안해도 그냥 if아니면 넘어가서 다음문장 실행이니까
        returns.append(row["text"])       #해당 인덱스(행)의 "text"를 문맥(returns)에 추가시켜준다.

    # 그렇게 결국 create_context는 문맥을 뱉어낸다.
    return "\n\n###\n\n".join(returns)  


# 이 함수가 하는 일 다시 정리해보자.
# 1. 질문이 들어오면 질문을 임베딩한다. 즉, 숫자 표현으로 바꿔준다.
# 2. 모든 문장들의 임베딩과 질문의 임베딩 사이의 거리를 계산한다.
# 3. 문맥이 너무 길어지면 안되니까, 가장 거리가 가까운 문장부터 차례대로 문맥에 추가한다.
# 4. 문맥을 반환한다.

In [ ]:
#create_context라는 함수는 인자로 question, df, max_len 총 세개를 가지나, max_len은 이미 1800으로 고정이다.
print(create_context('근로자란?', df))

In [ ]:


def answer_question(df, model="gpt-3.5-turbo", question, max_len=1800, debug =False):
    """
    데이터 프레임 텍스트에서 가장 유사한 문맥을 기반으로 질문에 답한다.
    """
    context = create_context(question, df, max_len=max_len)

    if debug:

        print("Context:\n" + context)
        print("\n\n")

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", "content": f"Context: {context}\n\n---\n\n Question: {question}"}
            ],
            temperature=0.5,
        )
        return response.choices[0].message.content
    except Exception as e:
        print("Error occurred:", e)
        return ""

In [ ]:
answer_question(df, question="근로자란?", debug=False)